In [52]:
import os
os.listdir()

['.ipynb_checkpoints',
 '.jupyter',
 'bottom_market_cap.csv',
 'company_financial_health_scores.xlsx',
 'data.csv',
 'new_project_ao.ipynb',
 'top_company_shortlist.csv',
 'top_csv_financials_sortedbymarketcap.csv',
 'top_market_cap.csv',
 'Updated_Project_FIle.ipynb',
 'Updated_Project_FIle.ipynb.invalid']

In [81]:
import pandas as pd
df = pd.read_csv("data.csv",usecols=['ticker','name','total_score','total_grade', 'industry'])

In [82]:
df.columns

Index(['ticker', 'name', 'industry', 'total_score', 'total_grade'], dtype='object')

In [89]:
df['industry']

0                                   Media
1                             Automobiles
2      Trading Companies and Distributors
3                       Consumer products
4                                   Media
                      ...                
717                             Machinery
718                           Real Estate
719                            Technology
720                               Energy 
721                   Aerospace & Defense
Name: industry, Length: 722, dtype: object

In [50]:
df['total_score'].describe()

count     722.000000
mean      975.750693
std       218.751796
min       600.000000
25%       763.000000
50%      1046.000000
75%      1144.000000
max      1536.000000
Name: total_score, dtype: float64

In [51]:
df['total_grade'].value_counts()


total_grade
BBB    368
B      167
BB     104
A       83
Name: count, dtype: int64

In [73]:
#filtering top ESG companies
score_threshold = df['total_score'].median()
acceptable_grades = ["A","AA","AAA"]

df_high_esg = df[
    (df["total_score"] >= score_threshold) &
    (df["total_grade"].isin(acceptable_grades))
].copy()

df_high_esg_sorted = df_high_esg.sort_values(by="total_score",ascending=False)

# df_high_esg_sorted[["ticker","name","industry","total_score","total_grade"]]
df_high_esg_sorted.shape[0]

83

In [74]:
df_high_esg_sorted.to_csv("top_company_shortlist.csv", index=False)

In [90]:
import yfinance as yf

top_esg_tickers = df_high_esg_sorted["ticker"]

def get_financials(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            "Ticker": ticker,
            "Sector": info.get("sector"),
            "marketCap": info.get("marketCap"),
            "Current Ratio": info.get("currentRatio"),
            "Quick Ratio": info.get("quickRatio"),
            "Debt to Equity": info.get("debtToEquity"),
            "Operating Margin": info.get("operatingMargins"),
            "Profit Margin": info.get("profitMargins"),
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return {"ticker": ticker}

# Fetch financials for top ESG companies
financial_data = [get_financials(ticker) for ticker in top_esg_tickers]

# Create a DataFrame from the collected data
financials_df = pd.DataFrame(financial_data)

#sorted by market cap
df_sorted_by_mc = financials_df.sort_values(by="marketCap",ascending=False)

# Save to CSV
df_sorted_by_mc.to_csv("top_csv_financials_sortedbymarketcap.csv", index=False)

In [102]:
# Optional preview
df_sorted_by_mc = df_sorted_by_mc.dropna() #drop rows with nulls
df_sorted_by_mc

,Ticker,Sector,marketCap,Current Ratio,Quick Ratio,Debt to Equity,Operating Margin,Profit Margin
1,msft,Technology,3.957345e+12,1.353,1.221,32.661,0.44901,0.36146
5,amzn,Consumer Cyclical,2.288595e+12,1.024,0.768,47.808,0.11432,0.10540
26,pg,Consumer Defensive,3.534195e+11,0.704,0.437,66.001,0.20848,0.18953
46,rtx,Industrials,2.091870e+11,1.006,0.605,67.855,0.11547,0.07352
7,pep,Consumer Defensive,1.912673e+11,0.775,0.559,276.868,0.17491,0.08229
...,...,...,...,...,...,...,...,...
75,clsk,Financial Services,2.975225e+09,8.668,0.923,34.234,-0.77185,-0.35434
53,he,Utilities,1.911234e+09,1.248,0.688,206.929,0.08512,-0.45445
56,abcl,Healthcare,1.279008e+09,10.146,8.863,6.632,-14.79575,0.00000
43,mtrx,Industrials,3.980053e+08,1.004,0.971,14.248,-0.02436,-0.03040


In [98]:
df_sorted_by_mc["Sector"].unique()

array(['Technology', 'Consumer Cyclical', 'Consumer Defensive',
       'Industrials', 'Healthcare', 'Utilities', 'Energy', 'Real Estate',
       'Basic Materials', 'Financial Services'], dtype=object)

In [99]:
industry_bench = {
    "Technology": {
        "profit_margin": 0.21,            # ~21% net margins for Info Tech sector :contentReference[oaicite:1]{index=1}
        "operating_margin": 0.21,
        "debt_to_equity": 0.65,           # typical IT range ~0.37‑0.92 :contentReference[oaicite:2]{index=2}
        "current_ratio": 2.0              # tech current ratio ~1.5‑3 :contentReference[oaicite:3]{index=3}
    },
    "Consumer Cyclical": {
        "profit_margin": 0.077,           # ~7.7% operating margin :contentReference[oaicite:4]{index=4}
        "operating_margin": 0.077,
        "debt_to_equity": 1.0,            # D/E typical range up to ~3.46 :contentReference[oaicite:5]{index=5}
        "current_ratio": 1.6              # retail & discretionary ~1.2‑1.8 :contentReference[oaicite:6]{index=6}
    },
    "Consumer Defensive": {
        "profit_margin": 0.069,           # consumer staples ~6.9% :contentReference[oaicite:7]{index=7}
        "operating_margin": 0.069,
        "debt_to_equity": 1.0,
        "current_ratio": 1.5
    },
    "Industrials": {
        "profit_margin": 0.106,           # industrials ~10.6% operating margin :contentReference[oaicite:8]{index=8}
        "operating_margin": 0.106,
        "debt_to_equity": 1.0,            # broad industrial range 0.28‑2.18 :contentReference[oaicite:9]{index=9}
        "current_ratio": 1.9              # industrials ~1.5‑2.4 :contentReference[oaicite:10]{index=10}
    },
    "Healthcare": {
        "profit_margin": 0.074,           # healthcare overall ~7.4% :contentReference[oaicite:11]{index=11}
        "operating_margin": 0.074,
        "debt_to_equity": 1.0,            # healthcare D/E ~0.53‑2.82 :contentReference[oaicite:12]{index=12}
        "current_ratio": 2.0              # typical ~1.5‑3 :contentReference[oaicite:13]{index=13}
    },
    "Utilities": {
        "profit_margin": 0.141,           # utilities net margins ~14.1% :contentReference[oaicite:14]{index=14}
        "operating_margin": 0.141,
        "debt_to_equity": 2.0,            # utilities often have high leverage ~1.08‑3.13 :contentReference[oaicite:15]{index=15}
        "current_ratio": 1.8              # utilities liquidity ~1.5‑2.0 :contentReference[oaicite:16]{index=16}
    },
    "Energy": {
        "profit_margin": 0.123,           # oil‑gas integrated ~12.3% :contentReference[oaicite:17]{index=17}
        "operating_margin": 0.123,
        "debt_to_equity": 1.3,            # energy D/E range ~0.29‑2.42 :contentReference[oaicite:18]{index=18}
        "current_ratio": 1.6              # capital‑intensive typical ~1.4‑2.0 :contentReference[oaicite:19]{index=19}
    },
    "Real Estate": {
        "profit_margin": 0.239,           # REIT net margin ~23.9% :contentReference[oaicite:20]{index=20}
        "operating_margin": 0.239,
        "debt_to_equity": 1.0,            # typical REIT ~0.54‑1.57 :contentReference[oaicite:21]{index=21}
        "current_ratio": 1.4              # REITs ~1.2‑1.6 :contentReference[oaicite:22]{index=22}
    },
    "Basic Materials": {
        "profit_margin": 0.101,           # materials ~10.1% operating profit margin :contentReference[oaicite:23]{index=23}
        "operating_margin": 0.101,
        "debt_to_equity": 0.9,            # materials D/E ~0.2‑1.29 :contentReference[oaicite:24]{index=24}
        "current_ratio": 1.8              # base materials ~1.6‑2.2 :contentReference[oaicite:25]{index=25}
    },
    "Financial Services": {
        "profit_margin": 0.135,           # non‑bank financial services ~13.5% :contentReference[oaicite:26]{index=26}
        "operating_margin": 0.135,
        "debt_to_equity": 2.0,            # financials up to ~4.5 but average ~2.0 :contentReference[oaicite:27]{index=27}
        "current_ratio": 1.5              # banks often around 1.3‑2.0 :contentReference[oaicite:28]{index=28}
    }
}

In [104]:
def score_ratio(company, benchmark, higher_higher=True):
    if company is None or pd.isna(company) or benchmark is None or pd.isna(benchmark):
        return 0
    if company == 0:
        return 0
    ratio = company / benchmark if higher_higher else benchmark / company
    return min(ratio, 2)


In [105]:
def calculate_scores(row):
    sector = row['Sector']
    if sector not in industry_bench:
        return pd.Series({
            'liquidity_score': 0,
            'solvency_score': 0,
            'op_margin_score': 0,
            'profit_margin_score': 0,
            'profit_score': 0,
            'financial_health_score': 0
        })

    bench = industry_bench[sector]

    liquidity_score = score_ratio(row['Current Ratio'], bench['current_ratio'])
    solvency_score = score_ratio(row['Debt to Equity'], bench['debt_to_equity'], higher_higher=False)
    op_margin_score = score_ratio(row['Operating Margin'], bench['operating_margin'])
    profit_margin_score = score_ratio(row['Profit Margin'], bench['profit_margin'])

    profit_score = (op_margin_score + profit_margin_score) / 2
    financial_health_score = (
        0.2 * liquidity_score +
        0.2 * solvency_score +
        0.6 * profit_score
    )

    return pd.Series({
        'liquidity_score': liquidity_score,
        'solvency_score': solvency_score,
        'op_margin_score': op_margin_score,
        'profit_margin_score': profit_margin_score,
        'profit_score': profit_score,
        'financial_health_score': financial_health_score
    })


In [106]:
df_sorted_by_mc = df_sorted_by_mc.join(df_sorted_by_mc.apply(calculate_scores, axis=1))

In [107]:
df_sorted_by_mc

,Ticker,Sector,marketCap,Current Ratio,Quick Ratio,Debt to Equity,Operating Margin,Profit Margin,liquidity_score,solvency_score,op_margin_score,profit_margin_score,profit_score,financial_health_score
1,msft,Technology,3.957345e+12,1.353,1.221,32.661,0.44901,0.36146,0.676500,0.019901,2.000000,1.721238,1.860619,1.255652
5,amzn,Consumer Cyclical,2.288595e+12,1.024,0.768,47.808,0.11432,0.10540,0.640000,0.020917,1.484675,1.368831,1.426753,0.988235
26,pg,Consumer Defensive,3.534195e+11,0.704,0.437,66.001,0.20848,0.18953,0.469333,0.015151,2.000000,2.000000,2.000000,1.296897
46,rtx,Industrials,2.091870e+11,1.006,0.605,67.855,0.11547,0.07352,0.529474,0.014737,1.089340,0.693585,0.891462,0.643720
7,pep,Consumer Defensive,1.912673e+11,0.775,0.559,276.868,0.17491,0.08229,0.516667,0.003612,2.000000,1.192609,1.596304,1.061838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,clsk,Financial Services,2.975225e+09,8.668,0.923,34.234,-0.77185,-0.35434,2.000000,0.058421,-5.717407,-2.624741,-4.171074,-2.090960
53,he,Utilities,1.911234e+09,1.248,0.688,206.929,0.08512,-0.45445,0.693333,0.009665,0.603688,-3.223050,-1.309681,-0.645209
56,abcl,Healthcare,1.279008e+09,10.146,8.863,6.632,-14.79575,0.00000,2.000000,0.150784,-199.942568,0.000000,-99.971284,-59.552613
43,mtrx,Industrials,3.980053e+08,1.004,0.971,14.248,-0.02436,-0.03040,0.528421,0.070185,-0.229811,-0.286792,-0.258302,-0.035260


In [118]:
# Merge ESG data (on Ticker)
final_esg_mc_df = df_sorted_by_mc[["Ticker", "Sector", "marketCap", "financial_health_score"]].merge(df[['ticker','name','total_score','total_grade']], left_on="Ticker", right_on="ticker",
                                                                                                     how="left")

# Assign Rank based on marketCap (already sorted)
final_esg_mc_df = final_esg_mc_df.reset_index(drop=True)
final_esg_mc_df["Rank"] = final_esg_mc_df.index + 1

# Reorder columns for PowerBI
cols = ["Rank", "Ticker", "name", "Sector", "marketCap", "total_score", "total_grade", "financial_health_score"]
final_esg_mc_df = final_esg_mc_df[cols]

# Export to CSV
final_esg_mc_df.to_csv("ranked_by_marketcap_with_esg.csv", index=False)


In [119]:
final_esg_mc_df

,Rank,Ticker,name,Sector,marketCap,total_score,total_grade,financial_health_score
0,1,msft,Microsoft Corp,Technology,3.957345e+12,1533,A,1.255652
1,2,amzn,Amazon.com Inc,Consumer Cyclical,2.288595e+12,1433,A,0.988235
2,3,pg,Procter & Gamble Co,Consumer Defensive,3.534195e+11,1292,A,1.296897
3,4,rtx,Raytheon Technologies Corp,Industrials,2.091870e+11,1250,A,0.643720
4,5,pep,PepsiCo Inc,Consumer Defensive,1.912673e+11,1417,A,1.061838
...,...,...,...,...,...,...,...,...
70,71,clsk,CleanSpark Inc,Financial Services,2.975225e+09,1206,A,-2.090960
71,72,he,Hawaiian Electric Industries Inc,Utilities,1.911234e+09,1243,A,-0.645209
72,73,abcl,Abcellera Biologics Inc,Healthcare,1.279008e+09,1239,A,-59.552613
73,74,mtrx,Matrix Service Co,Industrials,3.980053e+08,1256,A,-0.035260
